<a href="https://colab.research.google.com/github/hana927/News-Summarization/blob/main/News_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets spacy networkx transformers rouge-score nltk accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=81165b0c30e7b4c4cdb5372c272ad9cc01fb9f41deb08900519e769df1408c

# Load Data

The CNN/DailyMail dataset consists of approximately 300,000 pairs of news articles and their corresponding summaries.

In [2]:
from datasets import load_dataset,load_metric

In [3]:
cnn_data = load_dataset("cnn_dailymail", cache_dir='data',version="3.0.0")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Let's dive into it and take a closer look:

In [4]:
 cnn_data

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Given infrastructure constraints we subset our dataset and limit training on only 10000 records.

In [5]:
cnn_data['train'] = cnn_data['train'].shuffle(seed=42).select(range(10000))
cnn_data['validation'] = cnn_data['validation'].shuffle(seed=42).select(range(2000))
cnn_data['test'] = cnn_data['test'].shuffle(seed=42).select(range(2000))

 This database contains three sub-databases, named 'train', 'validation', and 'test', respectively.Given a key (train, validation, or test) and an index, you can view the data.

In [6]:
cnn_data['train'][6]

{'article': 'Every frontline police officer should be offered a Taser to help fight the threat from lone-wolf terrorists, a police leader declared yesterday. Steve White, who chairs the Police Federation, said evidence of plans to murder officers made the move necessary. ‘The terrorist ideal to get attention no longer relies on an attack being in a place of note,’ he said. ‘It could be in Cheam high street, in any town, in any part of the UK. We know there are more dangerous people out there, preparing to attack police officers and we need to be able to respond to that threat.’ Steve White, who chairs the Police Federation, said evidence of plans to murder officers made it necessary for every frontline officer to be offered a taser . Leaders of the Police Federation are set to vote next month about whether every uniformed officer should be offered training in the use of a Taser . The terrorist threat level for police was raised to severe two weeks ago in the wake of the Paris atrocity 

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(cnn_data["train"])

Since every article in this database has a manually written abstract, it makes it a good dataset for building an abstractive summarization model. Of course, if we only utilize the article portion, we can also use this database to build an extractive summarization model.

# Extractive Summarization

Extractive text summarization is the process of generating a summary of a longer document by selecting a subset of its sentences that convey the most important information. In here, I will use the SpaCy library to perform extractive text summarization in Python.

In [44]:
import spacy

In [45]:
import numpy as np
import networkx as nx

In [46]:

# Download and load the 'en_core_web_sm' model
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Preprocessing

Before we can summarize the text, we need to preprocess it by removing stop words and punctuation and lemmatizing the remaining words. We will also split the text into individual sentences for easier processing.

In [49]:

def preprocess_text(document):
    nlp = spacy.load("en_core_web_sm")

    # Preprocess the text
    doc = nlp(document)

    # Split the text into individual sentences
    sentences = [sent.text.strip() for sent in doc.sents]

    # Create a list of stopwords
    stopwords = list(nlp.Defaults.stop_words)

    # Remove stopwords and punctuation, and lemmatize the remaining words
    lemmatized_sentences = []
    for sentence in sentences:
        words = []
        for word in nlp(sentence):
            if not word.is_stop and not word.is_punct:
                words.append(word.lemma_)
        lemmatized_sentences.append(" ".join(words))

    return lemmatized_sentences


## TextRank Algorithm

To perform extractive text summarization, we will use the TextRank algorithm. The TextRank algorithm is based on the PageRank algorithm used by Google to rank web pages. In the context of text summarization, the TextRank algorithm treats sentences as nodes in a graph, and edges between sentences represent their similarity.

In [50]:
def generate_summary(lemmatized_sentences, num_sentences=3):
    # Calculate the similarity matrix
    similarity_matrix = []
    for i in range(len(lemmatized_sentences)):
        row = []
        for j in range(len(lemmatized_sentences)):
            row.append(nlp(lemmatized_sentences[i]).similarity(nlp(lemmatized_sentences[j])))
        similarity_matrix.append(row)

    # Convert the similarity matrix to a graph
    graph = nx.from_numpy_array(np.array(similarity_matrix))

    # Calculate the PageRank scores
    scores = nx.pagerank(graph)

    # Sort the sentences by their scores and extract the top N sentences
    top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
    summary = [lemmatized_sentences[i] for i in top_sentence_indices]
    summary = [sentence.capitalize() for sentence in summary]
    return ". ".join(summary)



We have used SpaCy to calculate the similarity between sentences based on their lemmatized text and converted the similarity matrix to a graph. We then used the PageRank algorithm from the NetworkX library to calculate the scores of each sentence in the graph and selected the top N sentences to include in the summary.

Let's try some examples in the CNN news dataset.

In [51]:
example_news= cnn_data['train'][0:3]['article']

In [52]:
extractive_summaries = []
for document in example_news:
    lemmatized_sentences = preprocess_text(document)
    num_sentences = 3
    summary = generate_summary(lemmatized_sentences, num_sentences)
    extractive_summaries.append(summary)

<ipython-input-50-a41568b68058>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  row.append(nlp(lemmatized_sentences[i]).similarity(nlp(lemmatized_sentences[j])))
<ipython-input-50-a41568b68058>:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  row.append(nlp(lemmatized_sentences[i]).similarity(nlp(lemmatized_sentences[j])))


In [53]:
for i, document in enumerate(example_news):
    print(f"Article:")
    print(document)
    print("Summary:")
    print(extractive_summaries[i])
    print()

Article:
By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detector i

# Abstractive Summarization

An abstractive summary is a type of text summarization where the summary is generated in a more human-like way. Unlike extractive summarization, where sentences are selected directly from the original text, abstractive summarization involves interpreting and paraphrasing the content of the input text to create a concise and coherent summary. It goes beyond simply extracting sentences and allows for more creativity in generating the summary.

## Pipeline

T5 and Distilbart are two very popular transformer models. Firstly let's use pipeline to take a brief look at the summarization quality of these two pre-train models.
Although the models we are exploring have different maximum input sizes, we will limit the input text to 2000 characters to ensure all models have the same input length for better comparability of the outputs.

In [16]:
from transformers import pipeline, set_seed
set_seed(42)

In [17]:
sample_text = cnn_data["train"][6]["article"][:2000]

A convention in summarization is to separate summary sentences with a newline. In here, we are using Punkt tokenizer in NLTK library.

In [18]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
summaries = {}

In [20]:
summaries["ground truth"]=cnn_data["train"][6]["highlights"]

### DistilBart

Distilbart CNN 12-6 is one of the most downloaded Hugging Face pre-trained models used for text summarization.

In [21]:
pipe = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
pipe_out = pipe(sample_text)
summaries["distilbart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### T5

The T5 Transformer (Text-To-Text Transfer Transformer) is a state-of-the-art language model developed by Google Research. It builds upon the foundation of the Transformer architecture and introduces several improvements to tackle various natural language processing tasks.

In [22]:
pipe = pipeline("summarization", model="t5-small")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

## Comparision

In [23]:
import pandas as pd

In [24]:
summaries_df=pd.DataFrame([summaries])

In [25]:
summaries_df

,ground truth,distilbart,t5
0,Calls for every police officer to be offered a...,Steve White said evidence of plans to murder ...,'the terrorist ideal to get attention no longe...


## Evaluation

In [26]:
metric = load_metric("rouge")

<ipython-input-26-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [27]:
reference = summaries["ground truth"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
for model_name in summaries:
	metric.add(prediction=summaries[model_name], reference=reference)
	score = metric.compute()
	rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
	records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())


,rouge1,rouge2,rougeL,rougeLsum
ground truth,1.000000,1.000000,1.000000,1.000000
distilbart,0.333333,0.170213,0.291667,0.312500
t5,0.363636,0.213333,0.337662,0.337662


We can see that the performance of T5 is slightly better than distilbart.

## Fine-Tuning T5

In [28]:
model_checkpoint = "t5-small"

### Preprocessing

Before feeding the data into the model, we need to preprocess the data. The pre-processing tool is called Tokenizer, which first tokenizes the input, then transforms the tokens into the corresponding token IDs needed in the pre-model, and then into the input format needed by the model.For data preprocessing purposes, we instantiate our tokenizer using the AutoTokenizer.from_pretrained method, and we use as_target_tokenizer to control the tokens corresponding to the targets.

In [29]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

As we are using checkpoints from a T5 pre-trained model, we need to check for special prefixes.T5 uses special prefixes to tell the model exactly what tasks to do. In here our prefix is 'summarize'.

In [30]:
prefix = "summarize: "

Now we can put everything together to form our preprocessing function. When we preprocess our samples, we'll also truncate=True to make sure our super long sentences are truncated. By default, we'll automatically padding for shorter sentences.

In [31]:
max_input_length = 1024
max_target_length = 128

In [32]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Next, we preprocess all samples in the dataset using the map function. We apply the preprocessing function to all samples.

In [33]:
tokenized_datasets = cnn_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Since the data is already prepared, now we need to download and load our pretrained model, and then fine-tune it. We use AutoModelForSeq2SeqLM for this purpose. Similar to the tokenizer, the from_pretrained method can help us download and load the model while also caching it, so we won't download the model again unnecessarily.







In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In order to obtain a Seq2SeqTrainer training tool, we need to set the training parameters with Seq2SeqTrainingArguments. This training configuration includes all the attributes that define the training process.

In [35]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "t5-finetuned-cnn",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=False,
    fp16= True

)

We need a data collator to feed our preprocessed inputs to the model.

In [36]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Setting up the Seq2SeqTrainer, there's one last thing left, which is defining the evaluation method. We use metrics for evaluation. Before feeding the model's predictions into the evaluation, we will also perform some post-processing of the data:

In [37]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Finally, pass all the parameters, data, and model to the Seq2SeqTrainer.

In [38]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Call the train method to perform fine-tuning training.

In [39]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.958200,1.719350,24.215100,11.736200,20.036000,22.796900,19.000000


TrainOutput(global_step=2500, training_loss=1.960752197265625, metrics={'train_runtime': 905.4003, 'train_samples_per_second': 11.045, 'train_steps_per_second': 2.761, 'total_flos': 2652677104336896.0, 'train_loss': 1.960752197265625, 'epoch': 1.0})

## Inference

In [40]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [41]:
pipe = pipeline(task='summarization', model=model, tokenizer=tokenizer)
pipe_out = pipe(sample_text)
summaries["finetunedt5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [42]:
summaries_df=pd.DataFrame([summaries])

In [43]:
summaries_df

,ground truth,distilbart,t5,finetunedt5
0,Calls for every police officer to be offered a...,Steve White said evidence of plans to murder ...,'the terrorist ideal to get attention no longe...,"Steve White, who chairs the Police Federation,..."
